In [1]:
!wget https://raw.githubusercontent.com/ChiThang-50Cent/review-classification-BERT/main/raw_data.csv

--2023-10-12 01:35:49--  https://raw.githubusercontent.com/ChiThang-50Cent/review-classification-BERT/main/raw_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 297379 (290K) [text/plain]
Saving to: ‘raw_data.csv’

raw_data.csv        100%[===================>] 290.41K  --.-KB/s    in 0.04s   

2023-10-12 01:35:49 (7.20 MB/s) - ‘raw_data.csv’ saved [297379/297379]



In [2]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.3 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4306 sha256=dfa952d85f787b6777196089372659ca0f6fdd14d884f2022531b124a8e774ea
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [3]:
import pandas as pd
import re
import py_vncorenlp

py_vncorenlp.download_model(save_dir='./')
segment_model = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='./')

In [4]:
df = pd.read_csv('./raw_data.csv')
df.sample(10)

,item_id,shop_id,comment,rate_star,label
1288,22077966900,76448301,nghe hay . bass sâu. chất lượng âm thanh hay. ...,5,0
783,23947047067,76448301,"Dùng thích, theo dõi shop và mua thêm nhiều mẫ...",5,0
880,16095114649,76448301,Tính năng nổi bật:nghe nhac\nChất lượng sản ph...,5,0
757,22437317005,76448301,"Tự sạc rất kém, chập chờn lúc được lúc k. Tai ...",2,0
965,19892939167,76448301,☆☆☆ Vậy nên mình đã có tâm sắp xếp thời gian c...,5,0
255,23179946587,76448301,Mua chính hãng nên yên tâm nhưng vì GHN làm vi...,5,0
1133,16569045493,76448301,NaN,5,0
199,21892301958,76448301,Sản phẩm giống như trong hình. Đẹp chất liệu t...,5,0
1481,19783359107,76448301,Sạc ip đúng 1 lần còn chuâ full pin đã hết pin...,1,1
1431,22877970102,76448301,Như lồn,1,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1646 entries, 0 to 1645
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_id    1646 non-null   int64 
 1   shop_id    1646 non-null   int64 
 2   comment    1509 non-null   object
 3   rate_star  1646 non-null   int64 
 4   label      1646 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 64.4+ KB


In [6]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1509 entries, 0 to 1508
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_id    1509 non-null   int64 
 1   shop_id    1509 non-null   int64 
 2   comment    1509 non-null   object
 3   rate_star  1509 non-null   int64 
 4   label      1509 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 59.1+ KB


In [7]:
df['label'].value_counts()

0    1248
1     261
Name: label, dtype: int64

In [8]:
df['rate_star'].value_counts()

5    1134
1     311
4      39
3      14
2      11
Name: rate_star, dtype: int64

In [9]:
df['comment'] = df['comment'].apply(lambda x: x.replace('\n', '. '))

In [10]:
df.loc[df['rate_star'] <= 2, 'label'] = 1
df.loc[df['rate_star'] >= 4, 'label'] = 0

In [11]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030" "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_special_characters(text):
    pattern = r"[^\w\s.,;:?]"
    return re.sub(pattern, "", text)

In [12]:
df['comment'] = df['comment'].apply(remove_emoji)
df['comment'] = df['comment'].apply(remove_special_characters)

In [13]:
df['comment'] = df['comment'].apply(lambda x: segment_model.word_segment(x)[0])

In [14]:
df = df.sort_values(by='rate_star')

In [16]:
df

,item_id,shop_id,comment,rate_star,label
1508,16223230656,76448301,Chất_lượng sản_phẩm : giao sai sản_phẩm .,1,1
1303,15597751010,76448301,Chất_lượng sản_phẩm : quá tệ dây lỗi chỉ dùng ...,1,1
1304,15597751010,76448301,"Mua xạc 3 đầu thì hư mất 2 đầu , : .",1,1
1305,15597751010,76448301,Xạc cho dòng idroi thì được còn xạc cho iphone...,1,1
1306,15597751010,76448301,Sản_phẩm không đúng không sạc nhanh chán .,1,1
...,...,...,...,...,...
512,21989879903,76448301,"Đúng với mô_tả : Đóng_gói chắc_chắn , sạc điện...",5,0
509,21989879903,76448301,"Đúng với mô_tả : Sạc nhanh nha , vừa chơi vừa ...",5,0
508,21989879903,76448301,Chất_lượng sản_phẩm : Tốt .,5,0
506,21989879903,76448301,"Đúng với mô_tả : Sạc nhanh thật_sự , giá sale ...",5,0


In [17]:
df.to_csv('./shopee_reviews_1.csv', index=False)